In [1]:
import ee
import geemap
from geemap import cartoee
import pandas as pd
import csv
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:1080'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:1080'
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
geemap.ee_user_id() 

'users/fxiahou'

In [3]:

roi_point1 = ee.FeatureCollection('users/fxiahou/Gas_flare/Timan_gas_54')
nature_point1 = ee.FeatureCollection('users/fxiahou/Gas_flare/Timan_nature_108')
roi_point2 = ee.FeatureCollection('users/fxiahou/Gas_flare/West_Siberia_gas__511')
nature_point2 = ee.FeatureCollection('users/fxiahou/Gas_flare/WSiberia_nature_746')
roi_point3 = ee.FeatureCollection('users/fxiahou/Gas_flare/East_Siberia_GAS_55')
nature_point3 = ee.FeatureCollection('users/fxiahou/Gas_flare/East_Siberia_nature_104')

In [4]:
Basin1 = ee.FeatureCollection('users/fxiahou/Basin/Timan_Pechora_Basin')
Basin2 = ee.FeatureCollection('users/fxiahou/Basin/West_Siberia_Basin')
Basin3 = ee.FeatureCollection('users/fxiahou/Basin/East_Siberia_Basin')
Basin4 = ee.FeatureCollection('users/fxiahou/Basin/Volga_Ural_Basin')
Basin5 = ee.FeatureCollection('users/fxiahou/Basin/Alaska_North_Slop')


In [5]:
roi_point = roi_point2
nature_point = nature_point2
Basin = Basin2.geometry()

In [6]:
dataset = ee.ImageCollection("MODIS/006/MOD11A2") \
    .filterDate('2000-01-01', '2006-12-31')
#rad = ee.List.sequence({start:1000, end:3000, step:1000})

# 这个温度单位是k，比例系数0.02.所以要先处理
#取时间

#取时间
def set_ssd(image) :
        time_start = image.get("system:time_start")
        image = image.set("system:time_start", time_start)
        image = image.multiply(0.02).subtract(273.15)
        return image

landSurfaceTemperatureVis = {'min': 14000.0,'max': 16000.0,\
    'palette': '040274, 040281, 0502a3, 0502b8, 0502ce, 0502e6,'\
    +'0602ff, 235cb1, 307ef3, 269db1, 30c8e2, 32d3ef,'\
    +'3be285, 3ff38f, 86e26f, 3ae237, b5e22e, d6e21f,'\
    +'fff705, ffd611, ffb613, ff8b13, ff6e08, ff500d,'\
    +'ff0000, de0101, c21301, a71001, 911003' }

In [7]:
Ts_impervious = ee.Image("Tsinghua/FROM-GLC/GAIA/v10").clip(Basin)
vis_impervious = { 'min': 0.0, 'max': 34, 'bands': ['change_year_index'], \
             'palette': '014352,1A492C,071EC4,B5CA36,729EAC,8EA5DE,818991,62A3C3,CCF4FE,74F0B9,32BC55,C72144,'\
                +'56613B,C14683,C31C25,5F6253,11BF85,A61B26,99FBC5,188AAA,C2D7F1,B7D9D8,856F96,109C6B,'\
                +'2DE3F4,9A777D,151796,C033D8,510037,640C21,31A191,223AB0,B692AC,2DE3F4' }
im_vector    = Ts_impervious.reduceToVectors( ee.Reducer.countEvery(), Basin,  550, 'polygon', False)
im_geo = im_vector.geometry()
im_geo.getInfo()
Map.addLayer(Ts_impervious, vis_impervious, "Change year index")
Map

{'geodesic': False,
 'type': 'MultiPolygon',
 'coordinates': [[[[63.11374540014094, 61.31307241312335],
    [63.118686134203585, 61.31307241312335],
    [63.118686134203585, 61.31801314718601],
    [63.11374540014094, 61.31801314718601],
    [63.11374540014094, 61.31307241312335]]],
  [[[63.1285676023289, 61.30813167906069],
    [63.13350833639157, 61.30813167906069],
    [63.13350833639157, 61.31307241312335],
    [63.1285676023289, 61.31307241312335],
    [63.1285676023289, 61.30813167906069]]],
  [[[63.1285676023289, 61.27848727468475],
    [63.13350833639157, 61.27848727468475],
    [63.13350833639157, 61.28342800874741],
    [63.1285676023289, 61.28342800874741],
    [63.1285676023289, 61.27848727468475]]],
  [[[63.1285676023289, 61.27354654062209],
    [63.13350833639157, 61.27354654062209],
    [63.13350833639157, 61.27848727468475],
    [63.1285676023289, 61.27848727468475],
    [63.1285676023289, 61.27354654062209]]],
  [[[63.138449070454215, 61.25872433843412],
    [63.143389

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [8]:
dataset1 = dataset.map(set_ssd)

In [9]:
def Set_rad_5000(Feature) :
     # var Loop_small = Feature.buffer(rad-1000)
        return Feature.buffer(5000)

In [10]:
#用油气点3000m进行掩模处理，那么后续就不用掩模
gas_rad =5000
Point_5000 = roi_point.map(Set_rad_5000)
Point_5000_geo = Point_5000.geometry()

In [11]:

#取缓冲区操作
def Set_rad(Feature) :
     # var Loop_small = Feature.buffer(rad-1000)
        return Feature.buffer(rad)
#选取范围、波段以及掩模 自然点需要掩模
def Set_Clip(image):
        data_clip = image.clip(Nature_rad).select('LST_Day_1km').updateMask(image_reduced.mask().Not()).updateMask(Ts_impervious.mask().Not())
        return data_clip
#计算范围内数据均值
def count_mean(img):
        Tem_mean = img.reduceRegion(**{
            'geometry':Nature_del_gas,
            'reducer':ee.Reducer.mean(),
            #'scale':250
        })
#        return NDVI_LOOPmean
        return img.set({"Tem_mean":Tem_mean})
#计算范围内数据纳入统计的总数
def count_point(img):
        point_number = img.reduceRegion(**{
            'geometry':Nature_del_gas,
            'reducer':ee.Reducer.count(),
            #'scale':250
        })
#        return NDVI_LOOPmean
        return img.set({"Point_number":point_number})

In [12]:
#点操作 
#选取该点、波段
def Set_Clip_point(image):
        data_clip = image.clip(nature_point).select('LST_Day_1km')
        return data_clip
#计算点内数据均值
def point_count_mean(img):
        Tem_mean = img.reduceRegion(**{
            'geometry':nature_point,
            'reducer':ee.Reducer.mean(),
            #'scale':250
        })
#        return NDVI_LOOPmean
        return img.set({"Tem_mean":Tem_mean})
#计算范围内数据纳入统计的总数
def point_count_point(img):
        point_number = img.reduceRegion(**{
            'geometry':nature_point,
            'reducer':ee.Reducer.count(),
            #'scale':250
        })
#        return NDVI_LOOPmean
        return img.set({"Point_number":point_number})

In [13]:
def Set_stop(Feature) :
     # var Loop_small = Feature.buffer(rad-1000)
        Nature_buffer =Feature.buffer(rad)
        Nature_buffer_area = Nature_buffer.area()
        Nature_buffer_8nu = ee.Number(Nature_buffer_area).multiply(0.9)
        Nature_5000gas = Nature_buffer.difference(Point_5000_geo)
        Nature_5000gas_area = Nature_5000gas.area()
        Nature_5000gas_nu = ee.Number(Nature_5000gas_area)
        num = ee.Algorithms.If(Nature_5000gas_nu.gt(Nature_buffer_8nu),Nature_5000gas,Feature)
        return num

In [16]:

test2= pd.DataFrame()
for rad in range(1000, 20001, 1000):
    Mod_time = []
    Ndvi_mean = []
    Ndvi_count = []
    #Nature_rad = nature_point.map(Set_rad)
    Nature_rad = nature_point.map(Set_stop)
    Nature_geo = Nature_rad.geometry()
    
    #这个是进行自然点减去油气点geometry对象，范围
    Nature_del_gas1 = Nature_geo.difference(Point_5000_geo)
    
    Nature_del_gas = Nature_del_gas1.difference(im_geo,50)
    print('success')
    #这个是油气点FeatureCollection对象
    Loop_mask = ee.FeatureCollection(Point_5000)
    #Map.addLayer(Nature_del_gas,{"color": "6755f1"},'{}rad'.format(str(rad)))
    image_reduced = Loop_mask.reduceToImage(['HasLabel'],'mean')
    rad
    
    data_filp = dataset1.map(Set_Clip).map(count_mean).map(count_point)
    Mod_time = data_filp.aggregate_array('system:index').getInfo()
    Tem_mean = data_filp.aggregate_array('Tem_mean').getInfo()
    Tem_count = data_filp.aggregate_array('Point_number').getInfo()
    #Ndvi_count
    test=pd.DataFrame(index=Mod_time,data=Tem_mean)
    test.columns = ['{}'.format(str(rad))+'Tem']
    test2 = pd.concat([test2,test], axis=1)
    
data_point_deal = dataset1.map(Set_Clip_point).map(point_count_mean).map(point_count_point)
Tem_mean_point = data_point_deal.aggregate_array('Tem_mean').getInfo()
Tem_count_point = data_point_deal.aggregate_array('Point_number').getInfo()
Tem_mean_point
Tem_count_point
point_ndvi_list = pd.DataFrame(index=Mod_time,data=Tem_mean_point)
point_ndvi_list.columns = ['point_Tem']
test2 = pd.concat([test2,point_ndvi_list], axis=1)

Map.addLayer(data_filp, landSurfaceTemperatureVis, '{}_tem'.format(str(rad)))
print(test2)
#test2.to_csv('D:/GEE/csv/test/null/Timan_NDVI.csv',encoding='utf-8')

Map.addLayer(Point_5000_geo,{"color": "6755f1"},'3000gas_rad')
Map.centerObject(Point_5000_geo, 8)
Map.addLayer(Loop_mask,{"color": "6755f1"},'{}rad'.format(str(rad)))
Map    

success


1000

success


2000

success


...

success


6000

success


7000

EEException: Computation timed out.

In [19]:
test2 = test2[[ 'point_Tem','1000Tem', '2000Tem', '3000Tem', '4000Tem', '5000Tem', '6000Tem', '7000Tem', '8000Tem',\
               '9000Tem', '10000Tem', '11000Tem', '12000Tem', '13000Tem', '14000Tem', '15000Tem', '16000Tem', \
               '17000Tem', '18000Tem', '19000Tem', '20000Tem']]
#去除存在空值 的行
No_null =test2.dropna()
#均值，axis=1时，为对行求均值
data_mean = pd.DataFrame(No_null.mean(axis=0))
data_mean
data_mean.plot()

,0
point_Tem,-2.838916
1000Tem,-2.826121
2000Tem,-2.824293
3000Tem,-2.821487
4000Tem,-2.822205
5000Tem,-2.824579
6000Tem,-2.828047
7000Tem,-2.829501
8000Tem,-2.828544
9000Tem,-2.825537


In [28]:
test2.to_csv('D:/GEE/test/Tem/West_Siberia_nature_tem.csv',header=1)
data_mean.to_csv('D:/GEE/test/Tem/West_Siberia_nature_tem_mean.csv',header=1)